# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [2]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [8]:
from sklearn.externals import joblib
eclf = joblib.load('../Entrega 5 Data Science/sentiment.pkl') 

In [10]:
eclf

VotingClassifier(estimators=[('svm', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)), ('random_forest', RandomForestClassifier(bootstrap=False, class_w...='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=500, random_state=None))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [11]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [12]:
wml_credentials={
  "apikey": "b6ZbbyP8bjQPg3e1-E5rwLPU-2QP1NMiqKjbpIFDfvIi",
  "iam_apikey_description": "Auto-generated for key f1099685-b3b2-4155-a333-49d8f7d12dd4",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/aa5ba3881e094eb3bfcd598610df0f49::serviceid:ServiceId-e711bb10-712f-4b18-9132-2db09fc1bd5b",
  "instance_id": "337b75aa-baff-4cb8-8b17-0eef6e466869",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [13]:
wml_credentials

{'apikey': 'b6ZbbyP8bjQPg3e1-E5rwLPU-2QP1NMiqKjbpIFDfvIi',
 'iam_apikey_description': 'Auto-generated for key f1099685-b3b2-4155-a333-49d8f7d12dd4',
 'iam_apikey_name': 'Credenciales de servicio-1',
 'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Writer',
 'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/aa5ba3881e094eb3bfcd598610df0f49::serviceid:ServiceId-e711bb10-712f-4b18-9132-2db09fc1bd5b',
 'instance_id': '337b75aa-baff-4cb8-8b17-0eef6e466869',
 'url': 'https://us-south.ml.cloud.ibm.com'}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [14]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [15]:
client

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [18]:
model_props = {
  client.repository.ModelMetaNames.NAME: "MovieReviewsClassification",
  client.repository.ModelMetaNames.DESCRIPTION: "Movies review NLP Classifier",
  client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20",
  client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3"}

In [20]:
model_props

{'name': 'MovieReviewsClassification',
 'description': 'Movies review NLP Classifier',
 'type': 'scikit-learn_0.20',
 'runtime': 'scikit-learn_0.20-py3'}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, eclf)
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth..., random_state=None))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [29]:
published_model= client.repository.store_model(
   model=pipeline,
   meta_props=model_props,
   training_data=X_train,
   training_target=y_train
)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [30]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [31]:
models_details = client.repository.list_models()

------------------------------------  --------------------------  ------------------------  -----------------
GUID                                  NAME                        CREATED                   TYPE
a515b194-8061-44d3-ac27-8ed1f6d4f9de  MovieReviewsClassification  2019-09-25T22:45:25.959Z  scikit-learn_0.20
cb867d62-c4a2-46fc-afa1-5f56cac0eb0f  MovieReviewsClassification  2019-09-25T22:43:31.940Z  scikit-learn_0.20
978f6615-33b7-4952-b911-6a079fa12726  MovieReviewsClassification  2019-09-25T22:41:23.424Z  scikit-learn_0.20
03fb1bae-24b4-4e0a-bc4e-4fdd35f7cab7  MovieReviewsClassification  2019-09-25T22:37:44.497Z  scikit-learn_0.20
------------------------------------  --------------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [32]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [33]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.855
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       200
           1       0.89      0.81      0.85       200

   micro avg       0.85      0.85      0.85       400
   macro avg       0.86      0.85      0.85       400
weighted avg       0.86      0.85      0.85       400

